In [11]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import event, create_engine

In [12]:
ny_crime = "NYC_crime.csv"
nycrime_df = pd.read_csv(ny_crime)
nycrime_df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,807539955,01/01/2016,12:00:00,01/24/2018,17:00:00,01/24/2018,361,OFF. AGNST PUB ORD SENSBLTY &,639,AGGRAVATED HARASSMENT 2,...,18,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,"988,085","217,902",40.764774,-73.986157,"(40.764774473000045, -73.986157253)"
1,908438691,01/01/2016,10:00:00,02/11/2018,16:00:00,02/11/2018,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",...,24,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,"993,373","229,301",40.796058,-73.967052,"(40.79605784000006, -73.96705230699997)"
2,713483285,01/01/2016,00:00:00,02/15/2018,19:30:00,02/16/2018,112,THEFT-FRAUD,739,"FRAUD,UNCLASSIFIED-FELONY",...,32,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,"1,001,374","238,163",40.820370,-73.938133,"(40.82036969600006, -73.93813284599996)"
3,178447094,01/01/2016,08:00:00,03/18/2017,17:00:00,02/21/2018,109,GRAND LARCENY,420,"LARCENY,GRAND BY OPEN/COMPROMISE CELL PHONE ACCT",...,34,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,"1,001,606","251,324",40.856492,-73.937261,"(40.85649236300002, -73.93726057999999)"
4,911003056,01/01/2016,12:00:00,02/16/2018,12:00:00,02/28/2018,109,GRAND LARCENY,407,"LARCENY,GRAND BY DISHONEST EMP",...,20,INSIDE,OTHER,NaN,NaN,"989,731","222,727",40.778017,-73.980211,"(40.77801697500007, -73.98021132399998)"


In [13]:
list(nycrime_df)

['CMPLNT_NUM',
 'CMPLNT_FR_DT',
 'CMPLNT_FR_TM',
 'CMPLNT_TO_DT',
 'CMPLNT_TO_TM',
 'RPT_DT',
 'KY_CD',
 'OFNS_DESC',
 'PD_CD',
 'PD_DESC',
 'CRM_ATPT_CPTD_CD',
 'LAW_CAT_CD',
 'JURIS_DESC',
 'BORO_NM',
 'ADDR_PCT_CD',
 'LOC_OF_OCCUR_DESC',
 'PREM_TYP_DESC',
 'PARKS_NM',
 'HADEVELOPT',
 'X_COORD_CD',
 'Y_COORD_CD',
 'Latitude',
 'Longitude',
 'Lat_Lon']

In [14]:
# Create a filtered dataframe from specific columns
crime_cols = [ "RPT_DT", "OFNS_DESC","Latitude", "Longitude"]
crime_trans= nycrime_df[crime_cols].copy()
crime_trans.head()

,RPT_DT,OFNS_DESC,Latitude,Longitude
0,01/24/2018,OFF. AGNST PUB ORD SENSBLTY &,40.764774,-73.986157
1,02/11/2018,HARRASSMENT 2,40.796058,-73.967052
2,02/16/2018,THEFT-FRAUD,40.820370,-73.938133
3,02/21/2018,GRAND LARCENY,40.856492,-73.937261
4,02/28/2018,GRAND LARCENY,40.778017,-73.980211


In [15]:
# Clean-up Date 
dates = []
dates = crime_trans['RPT_DT'].values
dates [1]

'02/11/2018'

In [19]:
# Get the Year
YYYY = []
for d in dates:
    d=str(d)
    YYYY.append(d[6:10])

In [20]:
# Get the Month
MM = []
for d in dates:
    MM.append(d[0:2])
print(MM)

['01', '02', '02', '02', '02', '03', '10', '10', '11', '12', '12', '01', '07', '05', '10', '01', '09', '03', '01', '01', '01', '07', '03', '04', '02', '02', '08', '06', '06', '02', '01', '01', '04', '02', '08', '09', '05', '03', '09', '10', '10', '03', '04', '05', '12', '10', '04', '03', '04', '04', '03', '04', '04', '11', '04', '04', '02', '02', '06', '06', '10', '10', '04', '01', '02', '03', '04', '10', '10', '01', '01', '12', '12', '06', '01', '01', '03', '03', '07', '07', '09', '05', '08', '01', '11', '01', '02', '06', '01', '01', '01', '01', '02', '01', '01', '01', '01', '01', '01', '06', '01', '06', '08', '06', '06', '08', '08', '01', '02', '03', '07', '03', '07', '10', '09', '05', '08', '08', '08', '02', '01', '02', '04', '08', '08', '08', '04', '01', '03', '08', '01', '08', '07', '08', '08', '01', '02', '03', '04', '05', '05', '05', '09', '12', '01', '03', '03', '05', '11', '12', '12', '03', '07', '05', '02', '02', '09', '08', '04', '04', '09', '01', '03', '04', '10', '12', '09

In [21]:
#Get the Day
DD = []
for d in dates:
    DD.append(d[3:5])
print(DD)

['24', '11', '16', '21', '28', '05', '09', '14', '12', '18', '19', '11', '02', '17', '16', '18', '01', '30', '28', '29', '30', '18', '22', '17', '16', '10', '27', '14', '14', '28', '31', '21', '04', '01', '06', '21', '23', '16', '29', '02', '01', '23', '26', '29', '25', '22', '09', '02', '06', '16', '20', '14', '14', '03', '17', '03', '13', '06', '25', '27', '11', '23', '20', '30', '09', '01', '03', '15', '18', '21', '27', '03', '06', '21', '10', '29', '22', '27', '02', '11', '18', '14', '11', '19', '11', '19', '06', '06', '19', '19', '19', '19', '21', '19', '19', '19', '19', '19', '19', '07', '29', '23', '04', '26', '26', '03', '16', '02', '09', '17', '24', '07', '06', '11', '27', '02', '10', '07', '27', '20', '29', '21', '04', '05', '06', '08', '28', '08', '13', '16', '31', '21', '11', '28', '29', '28', '12', '21', '11', '03', '22', '31', '08', '26', '15', '06', '22', '17', '06', '19', '04', '26', '03', '03', '01', '16', '26', '08', '02', '13', '19', '11', '27', '17', '16', '10', '07

In [22]:
crime_trans["Month"]=MM
crime_trans["Day"]=DD
crime_trans["Year"]=YYYY

In [61]:
final_crime = crime_trans[["Date","Year","Month", "Day", "OFNS_DESC","Latitude", "Longitude"]]

In [62]:
final_crime["Date"] = final_crime["Month"] + "-" + final_crime["Day"]
final_crime.head()

C:\Users\zhiyi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Date,Year,Month,Day,OFNS_DESC,Latitude,Longitude
0,01-24,2018,01,24,OFF. AGNST PUB ORD SENSBLTY &,40.764774,-73.986157
1,02-11,2018,02,11,HARRASSMENT 2,40.796058,-73.967052
2,02-16,2018,02,16,THEFT-FRAUD,40.820370,-73.938133
3,02-21,2018,02,21,GRAND LARCENY,40.856492,-73.937261
4,02-28,2018,02,28,GRAND LARCENY,40.778017,-73.980211


In [63]:
final_crime = final_crime.rename(index=str, columns={ 'OFNS_DESC':'Offense Description'})
final_crime.head()

,Date,Year,Month,Day,Offense Description,Latitude,Longitude
0,01-24,2018,01,24,OFF. AGNST PUB ORD SENSBLTY &,40.764774,-73.986157
1,02-11,2018,02,11,HARRASSMENT 2,40.796058,-73.967052
2,02-16,2018,02,16,THEFT-FRAUD,40.820370,-73.938133
3,02-21,2018,02,21,GRAND LARCENY,40.856492,-73.937261
4,02-28,2018,02,28,GRAND LARCENY,40.778017,-73.980211


In [65]:
#Create Database
connection_string = "root:19060Dst;@localhost/"
engine = create_engine(f'mysql://{connection_string}')

connection = engine.connect()

In [66]:
connection.execute('use etl_db;')

In [67]:
db_connection_string = "root:19060Dst;@localhost/etl_db"
db_engine = create_engine(f'mysql://{db_connection_string}')

# Confirm tables
db_engine.table_names()

['nyc_crime', 'nyc_crime_concat', 'nyc_parking']

In [68]:
final_crime.head()

,Date,Year,Month,Day,Offense Description,Latitude,Longitude
0,01-24,2018,01,24,OFF. AGNST PUB ORD SENSBLTY &,40.764774,-73.986157
1,02-11,2018,02,11,HARRASSMENT 2,40.796058,-73.967052
2,02-16,2018,02,16,THEFT-FRAUD,40.820370,-73.938133
3,02-21,2018,02,21,GRAND LARCENY,40.856492,-73.937261
4,02-28,2018,02,28,GRAND LARCENY,40.778017,-73.980211


In [69]:
final_crime.to_sql(name='nyc_crime', con=db_engine, if_exists='replace', index = False)

In [70]:
results = db_engine.execute('select * from nyc_crime')

In [71]:
for item in results:
    print(item)

('01-24', '2018', '01', '24', 'OFF. AGNST PUB ORD SENSBLTY &', 40.764774473, -73.986157253)
('02-11', '2018', '02', '11', 'HARRASSMENT 2', 40.7960578400001, -73.967052307)
('02-16', '2018', '02', '16', 'THEFT-FRAUD', 40.8203696960001, -73.938132846)
('02-21', '2018', '02', '21', 'GRAND LARCENY', 40.856492363, -73.93726058)
('02-28', '2018', '02', '28', 'GRAND LARCENY', 40.7780169750001, -73.980211324)
('03-05', '2018', '03', '05', 'SEX CRIMES', 40.8399082010001, -73.936083587)
('10-09', '2018', '10', '09', 'SEX CRIMES', 40.815732053, -73.945420411)
('10-14', '2018', '10', '14', 'THEFT-FRAUD', 40.7067067770001, -74.010933704)
('11-12', '2018', '11', '12', 'SEX CRIMES', 40.8087499040001, -73.9525858059999)
('12-18', '2018', '12', '18', 'THEFT-FRAUD', 40.7242001540001, -73.9772256429999)
('12-19', '2018', '12', '19', 'VEHICLE AND TRAFFIC LAWS', 40.731667658, -73.9894150869999)
('01-11', '2018', '01', '11', 'GRAND LARCENY', 40.8648942040001, -73.92407815)
('07-02', '2018', '07', '02', 'GRA